В интернет-галерее пользователи могут просматривать тысячи картинок произвольного содержания. При этом они совершают различные действия, например кликают на картинку чтобы увеличить её в размере или отложить её в избранное. Имея такую информацию, часто можно предугадать дальнейшие действия пользователя, на чём и основано большинство современных систем рекомендаций.

В этой задаче вам предлагается создать свою собственную систему рекомендаций.

Имея идентификаторы картинок, с которыми пользователи взаимодействовали, вам нужно предсказать, на какие картинки пользователи вероятнее всего кликнут. При этом картинки, на которые пользователи совершали клики в прошлом, не будут участвовать в оценке, даже если на них будет повторно совершён клик в будущем. Для предсказания множество картинок включает в себя все идентификаторы, которые представлены в файлах.



In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

from scipy.sparse import coo_matrix

import implicit

In [2]:
def load_df(filename, use_days=False):
    data = pd.read_csv('data/' + filename)
    if use_days:
        data.day = pd.to_datetime(data.day)
        data.day = data.day.max() - data.day
        data.day = (data.day / np.timedelta64(1, 'D')).astype(int)
        
    return data

def describe_df(data, name):
    print('------------{}------------'.format(name))
    print(data.shape)
    print(data.head())
    if 'day' in data.keys():
        print('last day is {}'.format(np.max(data['day'])))

def apk(actual, predicted, k=100): 
    """ 
    Computes the average precision at k. 
    This function computes the average prescision at k between two lists of 
    items. 
    Parameters 
    ---------- 
    actual : list 
             A list of elements that are to be predicted (order doesn’t matter) 
    predicted : list 
                A list of predicted elements (order does matter) 
    k : int, optional 
        The maximum number of predicted elements 
    Returns 
    ------- 
    score : double 
            The average precision at k over the input lists 
    """ 
    if len(predicted)>k: 
        predicted = predicted[:k] 
 
    score = 0.0 
    num_hits = 0.0 
 
    for i,p in enumerate(predicted): 
        if p in actual and p not in predicted[:i]: 
            num_hits += 1.0 
            score += num_hits / (i+1.0) 
 
    if not actual: 
        return 0.0 
 
    return score / min(len(actual), k) 

def mapk(actual, predicted, k=100): 
    """ 
    Computes the mean average precision at k. 
    This function computes the mean average prescision at k between two lists 
    of lists of items. 
    Parameters 
    ---------- 
    actual : list 
             A list of lists of elements that are to be predicted 
             (order doesn’t matter in the lists) 
    predicted : list 
                A list of lists of predicted elements 
                (order matters in the lists) 
    k : int, optional 
        The maximum number of predicted elements 
    Returns 
    ------- 
    score : double 
            The mean average precision at k over the input lists 
    """ 
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [13]:
def get_tags(pic_id):
    res = []
    for d in descriptions[descriptions.picture_id==pic_id].description:
        res = res + [int(tag) for tag in d.split()]
    return np.array(res)
    

def get_dstream(data_frames):
    """
    Compute sequence of user_id, image_id, date in 
    numpy array
    Parameters
    ---------- 
    data_frames : list
                  A list of dataframes with columns user_id and picture_id
    
    Returns 
    ------- 
    stream : numpy array (3, W)
            W is the total number of rows in data_frames
            stream[0] - user_id list
            stream[1] - image_id list
            stream[0] - date list
    """
    
    data = np.array([])
    X = np.array([])
    Y = np.array([])
    
    for df in data_frames:
        users = np.array(df.user_id)
        items = np.array(df.picture_id)
        days = np.array(df.day)
        
        data = np.hstack((data, days))
        X = np.hstack((X, users))
        Y = np.hstack((Y, items))
        
    stream = np.vstack((X, Y, data))
    return stream

def change_stream(stream):
    data = np.array([])
    X = np.array([])
    Y = np.array([])

    for user_id, picture_id, day in tqdm(stream.T):
        tags = get_tags(picture_id)
        users = np.ones_like(tags) * user_id
        days = np.ones_like(tags) * day
        
        data = np.hstack((data, days))
        X = np.hstack((X, users))
        Y = np.hstack((Y, tags))
    
    new_stream = np.vstack((X, Y, data))
    return new_stream

def create_st(data_frames, max_cost):
    """
    Create sparse table with different weights
    """
    
    stream = get_dstream(data_frames)
    print(stream[0].shape)
    print(stream[1].shape)
    print(stream[2].shape)
    stream[2] =  max_cost - stream[2] / 51. * (max_cost - 1)
    
    return coo_matrix((stream[2], (stream[0].astype(int), stream[1].astype(int))))

def create_st_desc(data_frames, max_cost):
    stream = change_stream(get_dstream(data_frames))
    stream[2] =  max_cost - stream[2] / 51. * (max_cost - 1)
    
    return coo_matrix((stream[2], (stream[0], stream[1])))

# Анализ данных

* ada

 ## Считываем данные

In [5]:
clicks = load_df('train_clicks.csv', True)
likes = load_df('train_likes.csv', True)
shares = load_df('train_shares.csv', True)
descriptions = load_df('descriptions.csv')
themes = load_df('themes.csv')
user_proﬁles = load_df('user_information.csv')
test_users = ('test_users.csv')

##  Выводим информацию

In [6]:
describe_df(clicks, 'clicks')
describe_df(likes, 'likes')
describe_df(shares, 'shares')
describe_df(descriptions, 'descriptions')
describe_df(themes, 'themes')
describe_df(user_proﬁles, 'user_proﬁles')

------------clicks------------
(2360862, 3)
   user_id  picture_id  day
0     1442      546149   43
1     1442      546149   43
2     1442     1242875   36
3     1442     1242875   36
4     1442     1242891   36
last day is 51
------------likes------------
(308422, 3)
   user_id  picture_id  day
0   490982      298754    3
1   490989     1060978   35
2   490989      976274   35
3   490989      976273   35
4   490989      976272   35
last day is 51
------------shares------------
(47867, 3)
   user_id  picture_id  day
0    24986      216354   37
1    25046      514369   16
2    25058     1066857   47
3    25058      481307   47
4    25058      636884   47
last day is 51
------------descriptions------------
(1379875, 2)
   picture_id                                        description
0     1171496      734475 487260 884706 5941 48255 147285 969800
1     1171497  488295 261184 466630 235887 642445 465419 9297...
2     1171498                              724435 1003828 164020
3     1171499

# Решение

In [85]:
def get_predictions(model, table_csr):
    rows = []
    for user_id in test_users.user_id.values:
        rows.append(model.recommend(user_id, table_csr, N=100))
    
    return rows

def concat_predictions(pred_list, weights):
    res = {}
    for i, pred in enumerate(pred_list):
        for img, val in pred:
            if img not in res.keys():
                res[img] = weights[i] * val
            else:
                res[img] += weights[i] * val
            
    return [i[0] for i in sorted(res.items(), key=lambda x: -x[1])[:100]]

In [ ]:
%%time
descriptions = descriptions.description.apply(pd.Series) \
    .merge(descriptions, right_index = True, left_index = True) \
    .drop(["description"], axis = 1) \
    .melt(id_vars = ['picture_id',], value_name = "tag") \
    .drop("variable", axis = 1) \
    .dropna()

Создаём разреженную матрицу пользователь — объект для трёх таблиц с разными весами

In [38]:
user_item = create_st([clicks, likes, shares], 8)
user_item_csr = user_item.tocsr()

(2717151,)
(2717151,)
(2717151,)


## Обучение моделей 

В качестве модели будем использовать разложение матрицы с помощью метода ALS

In [56]:
predictions = []

### Обучение крокодила

In [27]:
model = implicit.als.AlternatingLeastSquares(factors=512, regularization=.2, iterations=600, num_threads=0)

In [28]:
model.fit(user_item.T.tocsr())

100%|██████████| 600.0/600 [4:31:36<00:00, 27.75s/it]  


In [57]:
predictions.append(get_predictions(model, user_item_csr))

### Обучение помощника 1

In [40]:
np.random.seed(0)

In [41]:
help_model1 = implicit.als.AlternatingLeastSquares(factors=256, regularization=.1, iterations=400, num_threads=0)

In [42]:
help_model1.fit(user_item.T.tocsr())

100%|██████████| 400.0/400 [17:12<00:00,  2.61s/it]


In [58]:
predictions.append(get_predictions(help_model1, user_item_csr))

### Обучение помощника 2

In [114]:
np.random.seed(2)

In [115]:
help_model2 = implicit.als.AlternatingLeastSquares(factors=512, regularization=.1, iterations=200, num_threads=0)

In [116]:
help_model2.fit(user_item.T.tocsr())

100%|██████████| 200.0/200 [37:58<00:00, 10.35s/it]


In [117]:
predictions.append(get_predictions(help_model2, user_item_csr))

### Обучение помощника 3

In [81]:
np.random.seed(42)

In [82]:
help_model3 = implicit.als.AlternatingLeastSquares(factors=256, regularization=.2, iterations=500, num_threads=0)

In [83]:
help_model3.fit(user_item.T.tocsr())

100%|██████████| 500.0/500 [21:37<00:00,  2.58s/it]


In [84]:
predictions.append(get_predictions(help_model3, user_item_csr))

## Объединение результатов

In [145]:
rows = []
for i in range(len(predictions[0])):
    usr_pred_list = [pred[i] for pred in predictions]
    rows.append(concat_predictions(usr_pred_list, [1, 1, 0, 1, 1]))

## Сохранение результата

Отформатируем идентификаторы как нужно для ответа

In [146]:
test_users['predictions'] = list(map(lambda x: ' '.join(map(str, x)), rows))

И запишем предсказания в файл

In [147]:
test_users.to_csv('predictions.csv', index=False)

## Новый подход